# Intergrating action graph into the overall `Syft` workflow

In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
%pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)
from syft.service.action.action_graph_service import ActionGraphService
from syft.service.action.action_graph import ActionGraphStore
from syft.service.action.action_graph import InMemoryGraphConfig, InMemoryStoreClientConfig
from syft.service.context import AuthedServiceContext
from syft.node.credentials import SyftSigningKey
from syft.service.action.action_graph import Action
from syft.service.action.numpy import NumpyArrayObject, ActionObject
from syft.service.response import SyftError
from syft.types.syft_object import SyftObjectRegistry

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from pprint import pprint

print(sy.__version__)

## Non-mutation operations

Scenario for performing some computation


```python
import syft as sy

domain_client = sy.login("....")

a = domain_client.api.lib.numpy.array([1,2,3])

b = domain_client.api.lib.numpy.array([2,3,4])

c = a + b

d = domain_client.api.lib.numpy.array([1, 2, 3])

e = c * d
```

In [ ]:
node = sy.orchestra.launch(name="test-domain-ag-1", port='auto', node_side_type='low', dev_mode=True)
domain_client = node.login(email="info@openmined.org", password="changethis")

#### The client upload `a` and `b`

In [ ]:
a = domain_client.api.lib.numpy.array([1,2,3])
b = domain_client.api.lib.numpy.array([2,3,4])

In [ ]:
domain_client.api.services.graph.visualize()

In [ ]:
assert len(domain_client.api.services.graph.nodes()) == 6
assert len(domain_client.api.services.graph.edges()) == 4

#### The client adds `a` and `b`

In [ ]:
c = a + b

In [ ]:
domain_client.api.services.graph.visualize()

In [ ]:
assert len(domain_client.api.services.graph.nodes()) == 8
assert len(domain_client.api.services.graph.edges()) == 7

#### The client uploads `d`

In [ ]:
d = domain_client.api.lib.numpy.array([3,4,5])

In [ ]:
domain_client.api.services.graph.visualize()

In [ ]:
assert len(domain_client.api.services.graph.nodes()) == 11
assert len(domain_client.api.services.graph.edges()) == 9

#### `e = c * d`

In [ ]:
e = c * d

In [ ]:
domain_client.api.services.graph.visualize()

In [ ]:
assert len(domain_client.api.services.graph.nodes()) == 13
assert len(domain_client.api.services.graph.edges()) == 12

## Mutation operations (things are not working correctly for now)

```python
d[2] = 5 (mutation)
f = d + 48 (operating on the mutated object)
```

In [ ]:
# d[2] = 5
# domain_client.api.services.graph.visualize()
# print(domain_client.api.services.graph.nodes())
# f = d + 48
# domain_client.api.services.graph.visualize()

## Creating and uploading dataset

In [ ]:
node = sy.orchestra.launch(name="test-domain-action-graph-2", dev_mode=True, reset=True)
domain_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
num_assets = 2

dataset = sy.Dataset(name="Test Dataset")
dataset.set_description("""Test Dataset""")
dataset.add_citation("Person, place or thing")

country = sy.DataSubject(name="Country", aliases=["country_code"])
canada = sy.DataSubject(name="Canada", aliases=["country_code:ca"])
country.add_member(canada)
registry = domain_client.data_subject_registry
response = registry.add_data_subject(country)

for i in range(num_assets):
    data = pd.DataFrame(np.random.randint(0, 100, size=(10, 4)), 
                      columns=list('ABCD'))
    mock = pd.DataFrame(np.random.randint(0, 100, size=(10, 4)), 
                      columns=list('ABCD'))

    ctf = sy.Asset(name=f"test_dataset_{i}")
    ctf.set_description("""all the datas""")
    ctf.set_obj(data)
    ctf.set_shape((10, 4))
    ctf.add_data_subject(canada)
    ctf.set_mock(mock, mock_is_real=False)
    dataset.add_asset(ctf)

In [ ]:
domain_client.upload_dataset(dataset)

In [ ]:
domain_client.api.services.graph.visualize()

In [ ]:
assert len(domain_client.api.services.graph.nodes()) == num_assets
assert len(domain_client.api.services.graph.edges()) == 0

## Send an action object

In [ ]:
node = sy.orchestra.launch(name="test-domain-action-graph-3", dev_mode=True, reset=True)
domain_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
action_obj_a = ActionObject.from_obj([2, 4, 6])
action_obj_a.send(domain_client)

In [ ]:
action_obj_b = ActionObject.from_obj([1, 2, 3])
action_obj_b.send(domain_client)

In [ ]:
domain_client.api.services.graph.visualize()

In [ ]:
assert len(domain_client.api.services.graph.nodes()) == 2
assert len(domain_client.api.services.graph.edges()) == 0

## Calling `generate_remote_lib`

In [ ]:
node = sy.orchestra.launch(name="test-domain-action-graph-4", dev_mode=True, reset=True)
domain_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
a = domain_client.api.lib.numpy.array([1,2,3])
b = domain_client.api.lib.numpy.array([2,3,4])

In [ ]:
domain_client.api.services.graph.visualize()

In [ ]:
c = domain_client.api.lib.numpy.add(a, b)

In [ ]:
domain_client.api.services.graph.visualize()

In [ ]:
d = domain_client.api.lib.numpy.array([3,4,5])
e = domain_client.api.lib.numpy.multiply(c, d)

In [ ]:
domain_client.api.services.graph.visualize()

In [ ]:
assert len(domain_client.api.services.graph.nodes()) == 13
assert len(domain_client.api.services.graph.edges()) == 12

## Calling `syft_make_action`

In [ ]:
def helper_make_action_obj(orig_obj):
    obj_id = Action.make_id(None)
    lin_obj_id = Action.make_result_id(obj_id)

    return ActionObject.from_obj(orig_obj, id=obj_id, syft_lineage_id=lin_obj_id)


def helper_make_action_pointers(domain_client, obj, *args, **kwargs):
    obj_pointer = obj.send(domain_client)
    # The args and kwargs should automatically be pointerized by obj_pointer
    return obj_pointer, args, kwargs

In [ ]:
node = sy.orchestra.launch(name="test-domain-action-graph-5", dev_mode=True, reset=True)
domain_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
orig_obj, op, args, kwargs = ("abc", "find", ["b"], {})
# orig_obj, op, args, kwargs = ("abc", "capitalize", [], {})
# orig_obj, op, args, kwargs = (int(1), "__add__", [1], {})
# orig_obj, op, args, kwargs = ((1, 1, 3), "count", [1], {})


obj = helper_make_action_obj(orig_obj)
obj_pointer, args_pointers, kwargs_pointers = helper_make_action_pointers(
        domain_client, obj, *args, **kwargs
)
path = str(type(orig_obj))

In [ ]:
# action = obj.syft_make_action(path, op, remote_self= args=args_pointers, kwargs=kwargs_pointers)

In [ ]:
action = obj_pointer.syft_make_action(path, op, remote_self=obj_pointer.id, args=args_pointers, kwargs=kwargs_pointers)

In [ ]:
# print(domain_client.api.services.graph.nodes())
domain_client.api.services.graph.visualize()

In [ ]:
assert len(domain_client.api.services.graph.nodes()) == 5
assert len(domain_client.api.services.graph.edges()) == 4

In [ ]:
# Cleanup local domain server

if node.node_type.value == "python":
    node.land()